In [ ]:
import os
import glob
from shapely import wkt
from shapely.geometry import shape
from CopernicusAPI import S2_MSI_Catalogue, OData_Catalogue, CopernicusAccessToken, credentials_from_json
import zipfile
import json
import geojson
from osgeo import gdal

In [ ]:
year = "2023"

path = '../geodata/32vnm_latlng.geojson'
def from_json(path):
    with open(path) as f:
        features = json.load(f)["features"]
        for feature in features:
            geo = shape(feature["geometry"])
            # format geometry coordinates as WKT
            wkt = geo.wkt
            print(wkt)
            return wkt

def centroide(path):
    g = wkt.loads(from_json(path))
    print(g.centroid)
    return g.centroid

creds = credentials_from_json("./cop_creds.json") # .json file containing my credentials
CAT = CopernicusAccessToken(creds) # For authenticating

# Query = OData_Catalogue("SENTINEL-1")
Query = OData_Catalogue("Sentinel-2")

# Query.geographic(centroide(path))
Query.mrgs("32VNM") # Only works for Sentinel-2

Query = S2_MSI_Catalogue("L2A")
# Query.instrument("GRD")
Query.cloud_cover(0.3)
Query.start(f"{year}-06-01")
Query.end(f"{year}-09-01")

# Query.direct_filter(" and (Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'relativeOrbitNumber' and att/OData.CSC.DoubleAttribute/Value eq 44) \
#                     or Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'relativeOrbitNumber' and att/OData.CSC.DoubleAttribute/Value eq 139))")

Query.top(100)
# Query.orderby("start", "asc")

dl_folder = os.path.join("./Sentinel-2", year)
Query.Run(dl_folder, CAT)

In [ ]:
zips = glob.glob(os.path.join(dl_folder, '*.zip'))
for zip in zips:
    print(zip)
    with zipfile.ZipFile(zip, 'r') as z:
        z.extractall(dl_folder)
    os.remove(zip)

In [ ]:
import glob
years = [2020,2021,2022,2023]
for year in years:
    print(year)
    imgs = glob.glob(f'/filserver/frrov/imagery/Sentinel-1/*/*_aligned.tiff')
    print(len(imgs))
    # display(imgs)